In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210116.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice]
0,1/12/2021 17:46:10,Atlantic City (1980),Fatal Attraction (1987),Tickled (2016),[Default to Nicolas Cage],THE TRAIN IS THE STAR,VAMPIRES! SHIT!,WONG KAR- WHY? BECAUSE I GOTTA.
1,1/12/2021 18:45:16,Tickled (2016),Fatal Attraction (1987),Atlantic City (1980),VAMPIRES! SHIT!,[Default to Nicolas Cage],WONG KAR- WHY? BECAUSE I GOTTA.,THE TRAIN IS THE STAR
2,1/13/2021 8:48:10,Atlantic City (1980),Fatal Attraction (1987),NaN,THE TRAIN IS THE STAR,WONG KAR- WHY? BECAUSE I GOTTA.,VAMPIRES! SHIT!,[Default to Nicolas Cage]
3,1/13/2021 17:22:57,Tickled (2016),Atlantic City (1980),Fatal Attraction (1987),THE TRAIN IS THE STAR,WONG KAR- WHY? BECAUSE I GOTTA.,VAMPIRES! SHIT!,[Default to Nicolas Cage]
4,1/13/2021 19:04:00,Tickled (2016),Fatal Attraction (1987),Atlantic City (1980),THE TRAIN IS THE STAR,[Default to Nicolas Cage],VAMPIRES! SHIT!,WONG KAR- WHY? BECAUSE I GOTTA.
5,1/14/2021 8:27:39,Fatal Attraction (1987),Atlantic City (1980),Tickled (2016),WONG KAR- WHY? BECAUSE I GOTTA.,[Default to Nicolas Cage],THE TRAIN IS THE STAR,VAMPIRES! SHIT!
6,1/14/2021 15:33:54,Tickled (2016),Fatal Attraction (1987),Atlantic City (1980),VAMPIRES! SHIT!,WONG KAR- WHY? BECAUSE I GOTTA.,THE TRAIN IS THE STAR,[Default to Nicolas Cage]


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Atlantic City (1980)', 'Fatal Attraction (1987)',
       'Tickled (2016)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Atlantic City (1980)': <Candidate('Atlantic City (1980)')>,
 'Fatal Attraction (1987)': <Candidate('Fatal Attraction (1987)')>,
 'Tickled (2016)': <Candidate('Tickled (2016)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Atlantic City (1980), Fatal Attraction (1987), Tickled (2016))>,
 <Ballot(Tickled (2016), Fatal Attraction (1987), Atlantic City (1980))>,
 <Ballot(Atlantic City (1980), Fatal Attraction (1987))>,
 <Ballot(Tickled (2016), Atlantic City (1980), Fatal Attraction (1987))>,
 <Ballot(Tickled (2016), Fatal Attraction (1987), Atlantic City (1980))>,
 <Ballot(Fatal Attraction (1987), Atlantic City (1980), Tickled (2016))>,
 <Ballot(Tickled (2016), Fatal Attraction (1987), Atlantic City (1980))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

FINAL RESULT
Candidate                  Votes  Status
-----------------------  -------  --------
Tickled (2016)                 4  Elected
Atlantic City (1980)           2  Rejected
Fatal Attraction (1987)        1  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                          Votes  Status
-------------------------------  -------  --------
THE TRAIN IS THE STAR                  3  Hopeful
VAMPIRES! SHIT!                        2  Hopeful
WONG KAR- WHY? BECAUSE I GOTTA.        1  Rejected
[Default to Nicolas Cage]              1  Rejected

FINAL RESULT
Candidate                          Votes  Status
-------------------------------  -------  --------
THE TRAIN IS THE STAR                  5  Elected
VAMPIRES! SHIT!                        2  Rejected
WONG KAR- WHY? BECAUSE I GOTTA.        0  Rejected
[Default to Nicolas Cage]              0  Rejected

